<a href="https://colab.research.google.com/github/artemkush1/neuro/blob/master/Train_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

!pip install pymorphy2

import pandas as pd
import numpy as np
from numpy import isnan
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pymorphy2

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle as cPickle

def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true[y_true == 0] = 0.0001
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load data

In [0]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/bank_hack/preproc_train.csv", comment="#")

Y = data['sum_cltv']
data = data.drop(['sum_cltv'], axis=1)
data = data.drop(['mean_month_cltv'], axis=1)
data['sum_cltv'] = Y
data = data.values

# Show heatmap

In [0]:
def show_map(df_te_train):
  data1 = df_te_train.copy()
  corrmat = data1.corr()
  top_corr_features = corrmat.index
  plt.figure(figsize=(25,25))
  #plot heat map
  #print(data1[top_corr_features].shape)
  g=sns.heatmap(data1[top_corr_features].corr().abs(),annot=True,cmap="RdYlGn")


show_map(data)


# Train_test_split

In [0]:
X, x_, Y, y_ = train_test_split(data[:, :-1], data[:, -1], shuffle=True, test_size=0.2)
print(X.shape)
print(Y.shape)

(239752, 148)
(239752,)


# ***Models***

# GradientBoostingRegressor

*Test only one model*

In [0]:
gbr = GradientBoostingRegressor(n_estimators=150, max_depth=7)
gbr.fit(X, Y)
print(mean_absolute_error(Y, gbr.predict(X)))
print(mean_absolute_error(y_, gbr.predict(x_)))
print(mape(Y, gbr.predict(X)))
print(mape(y_, gbr.predict(x_)))


# save the classifier
with open('/content/drive/My Drive/Colab Notebooks/bank_hack/my_regres.pkl', 'wb') as fid:
    cPickle.dump(gbr, fid)

0.9312879907049695
1.2543248268716503
6587818618434940.0
2502.637578072109


*Test all models*

In [0]:
kf = KFold(n_splits=10, shuffle=True)
gbrs = []
final = 0
pred = 0
i = 0

for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  gbr = GradientBoostingRegressor(n_estimators=150, max_depth=7)
  gbr.fit(X_train, y_train)
  final += mean_absolute_error(y_test, gbr.predict(X_test))
  i += 1
  pred += mape(y_, gbr.predict(x_))
  print("curr score:", final / i, "curr pred score:", pred / i)
  with open(f'/content/drive/My Drive/Colab Notebooks/bank_hack/my_regres_{i}.pkl', 'wb') as fid:
    cPickle.dump(gbr, fid)  


print("final score:\n", final / i)

# LinearRegression

In [0]:
kf = KFold(n_splits=10, shuffle=True)
gbrs = []
final = 0
pred = 0
i = 0

for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  gbr = LinearRegression()
  gbr.fit(X_train, y_train)
  final += mean_absolute_error(y_test, gbr.predict(X_test))
  i += 1
  pred += mape(y_, gbr.predict(x_))  # mean_absolute_error(y_, gbr.predict(x_))
  print("curr score:", final / i, "curr pred score:", pred / i)
  with open(f'/content/drive/My Drive/Colab Notebooks/bank_hack/my_regres_{i + 10}.pkl', 'wb') as fid:
    cPickle.dump(gbr, fid)  


print("final score:\n", final / i)

curr score: 0.8490737805503066 curr pred score: 9529.29421624776
curr score: 0.8418087024696094 curr pred score: 9569.10916624418
curr score: 0.8450653299682264 curr pred score: 9845.251008401387
curr score: 0.8363663811541875 curr pred score: 9797.753469586116
curr score: 0.8327608110228599 curr pred score: 9663.494291919718
curr score: 0.8303776672175425 curr pred score: 9526.989719920275
curr score: 0.8295866575843897 curr pred score: 9579.064016894748
curr score: 0.8280660108666209 curr pred score: 9459.849013356716
curr score: 0.8275105185010542 curr pred score: 9452.698158565086
curr score: 0.8265895040619714 curr pred score: 9429.141365634132
final score:
 0.8265895040619714
